In [1]:
# pip install --upgrade pip
#!pip install pymysql

In [2]:
# pip install tensorflow==2.3.0

In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# 1) 데이터 로드하기

train_data = pd.read_table('../ratings_train.txt')
test_data = pd.read_table('../ratings_test.txt')

# 2) 데이터 정제하기

In [5]:
# train_data에 중복된 데이터 확인
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [6]:
train_data.drop_duplicates(subset=['document'], inplace=True)
# document 열에서 중복인 내용이 있다면 중복 제거

In [7]:
# 리뷰중에서 null값이 있는지 확인
train_data.isnull().values.any()

# 어떤 열에 null값이 존재하는지 확인
train_data.isnull().sum()
train_data.loc[train_data.document.isnull()]

train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data.isnull().values.any() # Null 값이 존재하는지 확인

False

In [8]:
# 데이터 전처리
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거

'do you expect people to read the FAQ etc and actually accept hard atheism'

In [9]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거

<ipython-input-9-6e6a69b985db>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [10]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)

<ipython-input-10-47938d712d24>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경


In [11]:
train_data.loc[train_data.document.isnull()][:5]

train_data = train_data.dropna(how = 'any')

In [12]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

<ipython-input-12-21a46579438d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
<ipython-input-12-21a46579438d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


# 3) 토큰화

In [13]:

# 불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [14]:
okt = Okt()
# okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

# stem = True
# 일정 수준의 정규화를 수행해준다

In [15]:
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)
    
# train_data 형태소 분석기를 사용하여 토큰화
# 불용어 제거후 X_train에 저장

In [16]:
X_test = []
for sentence in test_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

# 4) 정수 인코딩

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [18]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [19]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
# print('단어 집합의 크기 :',vocab_size)

In [20]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [21]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [22]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [23]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
#print(len(X_train))
#print(len(y_train))

C:\Users\yiseu\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [24]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [25]:
max_len = 30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.31944999380003


In [26]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [27]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [29]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [30]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1936/1936 [==============================] - 61s 31ms/step - loss: 0.3903 - acc: 0.8226 - val_loss: 0.3626 - val_acc: 0.8425

Epoch 00001: val_acc improved from -inf to 0.84249, saving model to best_model.h5
Epoch 2/15
1936/1936 [==============================] - 58s 30ms/step - loss: 0.3282 - acc: 0.8571 - val_loss: 0.3371 - val_acc: 0.8543

Epoch 00002: val_acc improved from 0.84249 to 0.85427, saving model to best_model.h5
Epoch 3/15
1936/1936 [==============================] - 60s 31ms/step - loss: 0.3030 - acc: 0.8719 - val_loss: 0.3286 - val_acc: 0.8587

Epoch 00003: val_acc improved from 0.85427 to 0.85868, saving model to best_model.h5
Epoch 4/15
1936/1936 [==============================] - 61s 31ms/step - loss: 0.2839 - acc: 0.8812 - val_loss: 0.3272 - val_acc: 0.8584

Epoch 00004: val_acc did not improve from 0.85868
Epoch 5/15
1936/1936 [==============================] - 63s 32ms/step - loss: 0.2679 - acc: 0.8904 - val_loss: 0.3290 - val_acc: 0.8609

Epoch 00005: 

In [43]:
model.save('best_model.h5')

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1527/1527 [==============================] - 9s 6ms/step - loss: 0.3787 - acc: 0.8489

 테스트 정확도: 0.8489
